<a href="https://colab.research.google.com/github/RajanMehta/nlp-pocs/blob/master/fuzzy_document_deduplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora
from gensim.similarities import Similarity

In [0]:
documents = [
    "Used SpaceX rocket as-is, buyer must transport.",
    "Used SpaceX rocket as-is, buyer must transport.",
    "For sale: bulk 100lbs pack of spaghetti noodles",
    "Spaghetti noodles for sale — 100lbs bulk pack",
    "Pale blue tuxedo, used, good condition. Call 555–555–5555",
    "Brand new yellow tuxedo in great condition!"
]

In [3]:
#Convert documents to collection of words
texts = [[text for text in simple_preprocess(doc, deacc=True)] for doc in documents]

#Build a bigram model to capture every pair of words in the texts
bigram = Phrases(texts, min_count=1)
bigram_phraser = Phraser(bigram)

#Reconvert documents to collection of words/bigrams
texts_bigrams = [[text for text in bigram_phraser[ simple_preprocess(doc, deacc=True)]] for doc in documents]

#Create dictionary
dictionary = corpora.Dictionary(texts_bigrams)

#Create corpus
corpus = [dictionary.doc2bow(docString) for docString in texts_bigrams]

#Build similarity index
index = Similarity(corpus=corpus,
                   num_features=len(dictionary),
                   output_prefix='on_disk_output')

#Parse similarities from index
doc_id = 0
similar_docs = {}
for similarities in index:
    similar_docs[doc_id] = list(enumerate(similarities))
    doc_id += 1

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:718: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/similarities/docsim.py:518: FutureWarnin

In [4]:
sim_threshold = 0.9

for doc_id, sim_doc_tuples in similar_docs.items():
  for sim_doc_tuple in sim_doc_tuples:
    sim_doc_id = sim_doc_tuple[0]
    sim_score = sim_doc_tuple[1]
    if (sim_score > sim_threshold) and (doc_id != sim_doc_id):
      print(f"Found similar documents, score of {sim_score:.2f}:")
      print('\t', documents[doc_id])
      print('\t', documents[sim_doc_id], '\n')

Found similar documents, score of 1.00:
	 Used SpaceX rocket as-is, buyer must transport.
	 Used SpaceX rocket as-is, buyer must transport. 

Found similar documents, score of 1.00:
	 Used SpaceX rocket as-is, buyer must transport.
	 Used SpaceX rocket as-is, buyer must transport. 

Found similar documents, score of 0.91:
	 For sale: bulk 100lbs pack of spaghetti noodles
	 Spaghetti noodles for sale — 100lbs bulk pack 

Found similar documents, score of 0.91:
	 Spaghetti noodles for sale — 100lbs bulk pack
	 For sale: bulk 100lbs pack of spaghetti noodles 

